In [3]:
!pip install pyspark

In [4]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Food DF').getOrCreate()

In [5]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [9]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [10]:
df = spark.read.csv('drive/My Drive/BDA-SPARK/indian_food.csv', header = True, inferSchema=True)
df.show(n=30)

+--------------+--------------------+----------+---------+---------+--------------+-------+-------------+----------+
|          name|         ingredients|      diet|prep_time|cook_time|flavor_profile| course|        state|    region|
+--------------+--------------------+----------+---------+---------+--------------+-------+-------------+----------+
|    Balu shahi|Maida flour, yogu...|vegetarian|       45|       25|         sweet|dessert|  West Bengal|      East|
|        Boondi|Gram flour, ghee,...|vegetarian|       80|       30|         sweet|dessert|    Rajasthan|      West|
|Gajar ka halwa|Carrots, milk, su...|vegetarian|       15|       60|         sweet|dessert|       Punjab|     North|
|        Ghevar|Flour, ghee, kewr...|vegetarian|       15|       30|         sweet|dessert|    Rajasthan|      West|
|   Gulab jamun|Milk powder, plai...|vegetarian|       15|       40|         sweet|dessert|  West Bengal|      East|
|        Imarti|Sugar syrup, lent...|vegetarian|       10|      

In [12]:
# 1. Find out how many unique dishes are present
df1=df.select('name').distinct()
df1.show()
df1.count()

+-----------------+
|             name|
+-----------------+
|            Kheer|
|          Poriyal|
|Sabudana Khichadi|
|   Gajar ka halwa|
|     Keerai sadam|
|         Dalithoy|
|         Mihidana|
|          Singori|
|          Uttapam|
|          Gavvalu|
|   Kakinada khaja|
|   Chak Hao Kheer|
|          Pachadi|
|         Vindaloo|
|     Gheela Pitha|
|         Idiappam|
|     Keerai kootu|
|             Idli|
|            Saath|
|            Rabri|
+-----------------+
only showing top 20 rows



255

In [15]:
# 2. Which state has more dishes?
state=df.select('state').groupBy('state').count()
state.select('state').filter(col('count')==state.agg({'count':'max'}).collect()[0][0]).collect()[0][0]

'Gujarat'

In [18]:
#  3. How many dishes from state Karnataka?
karDishes = df.select('name','state').filter(col('state') ==  "Karnataka").groupBy('name').count()
karDishes.count()

6

In [19]:
#  4. List number of unique regions
uniqueRegion = df.select('region').distinct().show()

+----------+
|    region|
+----------+
|        -1|
|     South|
|   Central|
|      East|
|      West|
|North East|
|     North|
|      NULL|
+----------+



In [22]:
#  5. Count number of dishes from each region.
dishes=df.select('name','region').groupBy('region').count()
dishes.show()

+----------+-----+
|    region|count|
+----------+-----+
|        -1|   13|
|      NULL|    1|
|     South|   59|
|   Central|    3|
|      East|   31|
|      West|   74|
|North East|   25|
|     North|   49|
+----------+-----+



In [24]:
# 6. List unique 'flavor_profile' and 'course'
flavor= df.select('flavor_profile','course').distinct().show()

+--------------+-----------+
|flavor_profile|     course|
+--------------+-----------+
|        bitter|      snack|
|         spicy|    starter|
|          sour|main course|
|            -1|      snack|
|            -1|main course|
|         sweet|main course|
|        bitter|main course|
|         spicy|      snack|
|         sweet|    dessert|
|         spicy|main course|
+--------------+-----------+



In [25]:
# 7. Which state has more 'main course'?
mainCourse = df.select('state','course').where(col('course') == 'main course').groupBy('state','course').count()
mainCourse.select('state','course').filter(col('count') == mainCourse.agg({'count':'max'}).collect()[0][0]).collect()[0][0]

'Punjab'

In [26]:
# 8. Give the %of dishes from each region.
percent = df.groupBy('region').count()
percent.withColumn('Percentage',(col('count')/percent.select(sum('count')).collect()[0][0])*100).orderBy(col('Percentage')).show()

+----------+-----+-------------------+
|    region|count|         Percentage|
+----------+-----+-------------------+
|      NULL|    1|0.39215686274509803|
|   Central|    3| 1.1764705882352942|
|        -1|   13|  5.098039215686274|
|North East|   25|  9.803921568627452|
|      East|   31| 12.156862745098039|
|     North|   49| 19.215686274509807|
|     South|   59| 23.137254901960784|
|      West|   74|  29.01960784313726|
+----------+-----+-------------------+



In [27]:
# 9. List the states which has more dishes from each region.
from pyspark.sql import Window
df.groupBy('region','state').count()\
        .withColumn('Rank',dense_rank().over(Window.partitionBy('Region').orderBy(col('count').desc())))\
        .filter(col('Rank')==1)\
        .orderBy(col('count'))\
        .show()

+----------+--------------+-----+----+
|    region|         state|count|Rank|
+----------+--------------+-----+----+
|      NULL| Uttar Pradesh|    1|   1|
|   Central|Madhya Pradesh|    2|   1|
|        -1|            -1|   13|   1|
|     South|    Tamil Nadu|   20|   1|
|North East|         Assam|   21|   1|
|      East|   West Bengal|   24|   1|
|     North|        Punjab|   32|   1|
|      West|       Gujarat|   35|   1|
+----------+--------------+-----+----+

